# Notebook
## Sistem Rekomendasi Obat berdasarkan _Weighted Hybrid Approach_

Notebook ini menyajikan penerapan dan eksperimentasi _weighted hybrid approach_ pada sistem rekomendasi obat. _Weighted hybrid approach_ yang diusulkan merupakan kombinasi antara .... dan .... . Beberapa teknik pembobotan yang akan dimasukkan dalam eksperimentasi antara lain ...., ...., dan .....

## 1. Pustaka

Impor pustaka yang dibutuhkan.

In [1]:
# pustaka eksternal
import pandas as pd
import plotly.graph_objects as go

# pustaka custom
from analysis import *

## 2. Persiapan Set Data

Pada bagian ini, dilakukan persiapan set data yang dibutuhkan dalam penelitian. Karena set data yang sesuai untuk secara langsung digunakan dalam penelitian ini belum tersedia saat penelitian ini dikerjakan, maka Peneliti melakukan pengumpulan data dari beberapa sumber dan menggabungkannya.

Set data `WebMD` dan `DailyMed` digabungkan dalam struktur data DataFrame.

In [3]:
# berkas WebMD Drug Reviews oleh Rohan Harode dari Kaggle
path_wmd = 'D:\JUPYTER NOTEBOOK\TUGAS AKHIR\data\webmd.csv'

# berkas DailyMed Drug dari situs web DailyMed
# yang dikompilasi dengan menggunakan generate_dailymed.py
path_dm = 'D:\JUPYTER NOTEBOOK\TUGAS AKHIR\data\dailyMed.csv'

df_wmd = pd.read_csv(path_wmd)
df_dm = pd.read_csv(path_dm)

Atribut yang diperlukan sebagai pelengkap data pada WebMD, antara lain:
1. nama obat secara umum pada kolom `sub_name`
2. daftar bahan aktif pada kolom `list_activeIngredient`, dan 
3. daftar bahan inaktif pada kolom `list_activeIngredient`.

Oleh karena itu, perlu dilakukan seleksi kolom pada DataFrame `df_dm` sebagai berikut.

In [5]:
df_dm = df_dm[["SubName", "ListActiveIngredient", "ListInactiveIngredient"]]

Gabungkan DataFrame DailyMed `df_dm` dan DataFrame MebMD `df_wmd` dengan menggunakan kolom `sub_name` di `df_dm` dan kolom `Drug` di `df_wmd` sebagai indeks pengabungan.

In [7]:
df_drugs = pd.merge(df_dm, df_wmd, left_on = "SubName", right_on = "Drug")

## 3. Analisis Data

Dalam penelitian ini, analisis dilakukan pada `df_drugs` untuk memahami tipe data dan mengidentifikasi kesalahan. Informasi tersebut digunakan untuk menentukan prosedur yang sesuai dalam tahap prapemrosesan.

Kita mulai dengan menghitung jumlah masing-masing nilai yang berbeda pada kolom `Drug` untuk mengetahui jumlah baris data untuk setiap obat. Informasi ini diperlukan untuk memastikan kecukupan data penelitian.

In [8]:
display(df_drugs["Drug"].value_counts())

lisinopril             12807
simvastatin             7203
benzonatate             5999
meloxicam               5394
prednisone              5152
                       ...  
zolmitriptan               5
sevelamer carbonate        3
bexarotene                 2
caffeine citrate           2
cromolyn sodium            2
Name: Drug, Length: 74, dtype: int64

Pada sel tersebut, kita dapat melihat rentang nilai dari setiap atribut. untuk attribut sub_name: 74 kategori, List_activeIngredient: 137 kategori, list_inactiveIngredient: 132 kategori, Age: 11 kategori, Condition: 405 kategori, Date: 3884 kategori, Drug:74 kategori, DrugId: 74 kategori, EaseofUse: 5 kategori, Effectiveness: 5 kategori, Reviews: 16967 kategori, Satisfaction: 5 kategori, Sex: 2 kategori, Sides: 69 kategori, UsefulCount: 94 kategori. 

## Type Data

In [14]:
df_drugs.dtypes

SubName                   object
ListActiveIngredient      object
ListInactiveIngredient    object
Age                       object
Condition                 object
Date                      object
Drug                      object
DrugId                     int64
EaseofUse                  int64
Effectiveness              int64
Reviews                   object
Satisfaction               int64
Sex                       object
Sides                     object
UsefulCount                int64
dtype: object

Untuk memeriksa tipe data dari masing2 varibael/kolom/fitur dengan menggunakan .dtypes. untuk dtype (object) pada pandas digunakan untuk tipe data Teks atau campuran nilai numerik dan non-numeric. untuk dtype (int64) pada pandas digunakan untuk tipe data Integer numbers. Pada sel diatas, kita dapat melihat bahwa 2 jenis type data yang muncul yaitu object dan int64.

In [15]:
for col in df_drugs:
    s = pd.get_dummies(df_drugs[col],prefix='', prefix_sep='').sum() 
    print(col, ":", s)

SubName : acarbose           24
acetazolamide     285
aripiprazole      477
baclofen          550
benzonatate      5999
                 ... 
valproic acid     124
voriconazole       33
zaleplon           82
zolmitriptan        5
zonisamide        202
Length: 74, dtype: int64
ListActiveIngredient : [['ACARBOSE', '25', 'mg', '1', '1']]           24
[['ACETAZOLAMIDE', '125', 'mg', '1', '1']]     95
[['ACETAZOLAMIDE', '250', 'mg', '1', '1']]     95
[['ACETAZOLAMIDE', '500', 'mg', '1', '1']]     95
[['ARIPIPRAZOLE', '1', 'mg', '1', 'mL']]       53
                                             ... 
[['linezolid', '2', 'mg', '1', 'mL']]           4
[['lovastatin', '10', 'mg', '1', '1']]        168
[['metolazone', '2.5', 'mg', '1', '1']]        27
[['ofloxacin', '3', 'mg', '1', 'mL']]           9
[['valproic acid', '250', 'mg', '1', '1']]     62
Length: 137, dtype: int64
ListInactiveIngredient : [['ACETIC ACID', '', '', '', ''], ['SODIUM CHLORIDE', '', '', '', ''], ['SODIUM ACETATE', '', '', '

Date : 1/1/2008    29
1/1/2009    34
1/1/2010    26
1/1/2011    18
1/1/2012    10
            ..
9/9/2013    16
9/9/2014     5
9/9/2015    20
9/9/2016    10
9/9/2017     5
Length: 3884, dtype: int64
Drug : acarbose           24
acetazolamide     285
aripiprazole      477
baclofen          550
benzonatate      5999
                 ... 
valproic acid     124
voriconazole       33
zaleplon           82
zolmitriptan        5
zonisamide        202
Length: 74, dtype: int64
DrugId : 676         96
911       5394
1027       114
1049       265
1636       352
          ... 
64437      477
93290      324
94574       23
148989     140
149812       3
Length: 74, dtype: int64
EaseofUse : 1     6681
2     3137
3     7106
4    12583
5    32162
dtype: int64
Effectiveness : 1    12037
2     6225
3    11020
4    13830
5    18557
dtype: int64
Reviews : \n                                                                                                                                                        

Pada sel tersebut, kita dapat melihat bahwa ada 74 nama obat dalam `df_drugs`. Obat dengan baris data terbanyak adalah `lisinopril` (12.807), sementara obat dengan baris data tersedikit adalah `cromolyn sodium`, `caffeine citrate`, dan `bexarotene` (2).

Jika menggunakan 1000 baris data sebagai ambang batas kecukupan data, maka persentase jumlah obat disajikan sebagai berikut.

In [10]:
n_sufficient = sum(x >= 1000 for x in df_drugs["Drug"].value_counts())
n_insufficient = sum(x < 1000 for x in df_drugs["Drug"].value_counts())
fig = go.Figure(data=[go.Pie(labels=["Cukup", "Tidak Cukup"],
                             values=[n_sufficient, n_insufficient], hole = 0.5)])
fig.show()

Memperhatikan persentase tersebut, maka diperlukan penambahan lebih banyak baris data untuk setiap obat dengan total baris data yang kurang dari 500. Untuk saat ini, kita cukup gunakan baris data obat yang cukup.

In [11]:
list_sufficient = [val for val, cnt in df_drugs["Drug"].value_counts().iteritems() if cnt >= 500]
df_drugs_sub = df_drugs[df_drugs["Drug"].isin(list_sufficient)]

Untuk menerapkan _item-based collaborative filtering_ pada sistem rekomendasi obat, dibutuhkan matriks pengguna/obat. Pada matriks tersebut terdapat _"user id"_, _"drug id"_, dan _"drug rating"_. Ilustrasi matriks tersebut adalah sebagai berikut.

|  | drug<sub>1</sub> | drug<sub>2</sub> | drug<sub>3</sub> |
| :---: | :---: | :---: | :---: |
| user<sub>1</sub> | rating<sub>user<sub>1</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>3</sub></sub> |
| user<sub>2</sub> | rating<sub>user<sub>2</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>3</sub></sub> |
| user<sub>3</sub> | rating<sub>user<sub>3</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>3</sub></sub> |
| user<sub>4</sub> | rating<sub>user<sub>4</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>3</sub></sub> |
| user<sub>5</sub> | rating<sub>user<sub>5</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>3</sub></sub> |


_"user id"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, ....

_"drug id"_ tersedia dalam `df_drugs`. Oleh karena itu, ....

_"drug rating"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, ....

In [13]:
df_drugs_sub.pivot_table(columns=["Sex", "Age", "Condition"], aggfunc="size")

AttributeError: 'Int64Index' object has no attribute 'remove_unused_levels'

## Mengidentifikasi Kesalahan 

In [11]:
df_drugs_sub.isnull().sum()

sub_name                    0
list_activeIngredient       0
list_inactiveIngredient     0
Age                         0
Condition                   0
Date                        0
Drug                        0
DrugId                      0
EaseofUse                   0
Effectiveness               0
Reviews                    16
Satisfaction                0
Sex                         0
Sides                       0
UsefulCount                 0
dtype: int64

Dalam mengidentifikasi kesalahahan dapat dilakukan agregasi data  menggunakan fungsi sum(). Berdasarkan analisis diatas dapat dilihat bahwa dalam dataset tersebut semua kolom tidak memiliki nilai kosong atau NULL kecuali kolom Reviews dengan 16 missing value.

In [12]:
data1 = df_drugs.drop(columns = ['Reviews'])
for col in data1:
    s = pd.get_dummies(df_drugs[col],prefix='', prefix_sep='').sum() 
    print(col, ":", s)

# s = pd.get_dummies(data1,prefix='', prefix_sep='').sum().sort()
# s


sub_name : acarbose           24
acetazolamide     285
aripiprazole      477
baclofen          550
benzonatate      5999
                 ... 
valproic acid     124
voriconazole       33
zaleplon           82
zolmitriptan        5
zonisamide        202
Length: 74, dtype: int64
list_activeIngredient : [['ACARBOSE', '25', 'mg', '1', '1']]           24
[['ACETAZOLAMIDE', '125', 'mg', '1', '1']]     95
[['ACETAZOLAMIDE', '250', 'mg', '1', '1']]     95
[['ACETAZOLAMIDE', '500', 'mg', '1', '1']]     95
[['ARIPIPRAZOLE', '1', 'mg', '1', 'mL']]       53
                                             ... 
[['linezolid', '2', 'mg', '1', 'mL']]           4
[['lovastatin', '10', 'mg', '1', '1']]        168
[['metolazone', '2.5', 'mg', '1', '1']]        27
[['ofloxacin', '3', 'mg', '1', 'mL']]           9
[['valproic acid', '250', 'mg', '1', '1']]     62
Length: 137, dtype: int64
list_inactiveIngredient : [['ACETIC ACID', '', '', '', ''], ['SODIUM CHLORIDE', '', '', '', ''], ['SODIUM ACETATE', '', ''

Condition :                                                            11
"Change of Life" Signs                                     17
A Condition of Bladder Dysfunction from Nerve Disorder      2
A Fungal Infection - Aspergillosis                          6
A Hereditary Progressive Anemia of Unknown Cause           18
                                                         ... 
Wide-Angle Glaucoma                                         2
Worsening of Asthma                                        66
Yeast Infection of Vagina and Vulva                       460
infection                                                   3
infection caused by bacteria                               19
Length: 405, dtype: int64
Date : 1/1/2008    29
1/1/2009    34
1/1/2010    26
1/1/2011    18
1/1/2012    10
            ..
9/9/2013    16
9/9/2014     5
9/9/2015    20
9/9/2016    10
9/9/2017     5
Length: 3884, dtype: int64
Drug : acarbose           24
acetazolamide     285
aripiprazole      477
baclofen

Berdasarkan cell code diatas, tidak ditemukan noise atau nilai aneh/ganjil untuk setiap field atau kolom. Namun terdapat nilai yang kosong seperti pada atribut 'Age'

## Membuat DataFrame

In [16]:
df_drugs.columns

Index(['SubName', 'ListActiveIngredient', 'ListInactiveIngredient', 'Age',
       'Condition', 'Date', 'Drug', 'DrugId', 'EaseofUse', 'Effectiveness',
       'Reviews', 'Satisfaction', 'Sex', 'Sides', 'UsefulCount'],
      dtype='object')

In [17]:
df_drugs.rename(columns={"DrugId": "ItemId"}, inplace = True)
df_drugs.columns

Index(['SubName', 'ListActiveIngredient', 'ListInactiveIngredient', 'Age',
       'Condition', 'Date', 'Drug', 'ItemId', 'EaseofUse', 'Effectiveness',
       'Reviews', 'Satisfaction', 'Sex', 'Sides', 'UsefulCount'],
      dtype='object')

In [18]:
# Data Drug
df_item = pd.DataFrame({'ItemId':df_drugs['ItemId'],'Drug':df_drugs['Drug'], 'ListActiveIngredient':df_drugs['ListActiveIngredient'], 'ListInactiveIngredient':df_drugs['ListInactiveIngredient'],'Sides':df_drugs['Sides']})
df_item

,ItemId,Drug,ListActiveIngredient,ListInactiveIngredient,Sides
0,6054,isosorbide mononitrate,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...","Headache , dizziness , lightheadedness , na..."
1,6054,isosorbide mononitrate,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...","Headache , dizziness , lightheadedness , na..."
2,6054,isosorbide mononitrate,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...","Headache , dizziness , lightheadedness , na..."
3,6054,isosorbide mononitrate,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...","Headache , dizziness , lightheadedness , na..."
4,6054,isosorbide mononitrate,"[['ISOSORBIDE MONONITRATE', '10', 'mg', '1', '...","[['LACTOSE, UNSPECIFIED FORM', '', '', '', '']...","Headache , dizziness , lightheadedness , na..."
...,...,...,...,...,...
61664,93290,fenofibrate,"[['FENOFIBRIC ACID', '160', 'mg', '1', '1']]","[['SILICON DIOXIDE', '', '', '', ''], ['CROSCA...",
61665,93290,fenofibrate,"[['FENOFIBRIC ACID', '160', 'mg', '1', '1']]","[['SILICON DIOXIDE', '', '', '', ''], ['CROSCA...",
61666,93290,fenofibrate,"[['FENOFIBRIC ACID', '160', 'mg', '1', '1']]","[['SILICON DIOXIDE', '', '', '', ''], ['CROSCA...",
61667,93290,fenofibrate,"[['FENOFIBRIC ACID', '160', 'mg', '1', '1']]","[['SILICON DIOXIDE', '', '', '', ''], ['CROSCA...",


In [19]:
# Data User
df_user = pd.DataFrame({'Age':df_drugs['Age'], 'Sex':df_drugs['Sex'], 'Condition':df_drugs['Condition']})

In [20]:
df_user['UserId'] = df_user.index
print(df_user)

              Age     Sex                                          Condition  \
0      75 or over    Male                                              Other   
1           55-64  Female                                              Other   
2      75 or over    Male                         Angina Pectoris Prevention   
3      75 or over    Male                         Angina Pectoris Prevention   
4      75 or over    Male                         Angina Pectoris Prevention   
...           ...     ...                                                ...   
61664       65-74          Combined High Blood Cholesterol and Triglyceri...   
61665       55-64    Male                                Low HDL Cholesterol   
61666       55-64    Male           High Amount of Triglyceride in the Blood   
61667       55-64  Female  Combined High Blood Cholesterol and Triglyceri...   
61668       35-44  Female  Combined High Blood Cholesterol and Triglyceri...   

       UserId  
0           0  
1      

In [21]:
df_user = pd.DataFrame({'UserId':df_user['UserId'], 'Age':df_drugs['Age'], 'Sex':df_drugs['Sex'], 'Condition':df_drugs['Condition']})
df_user

,UserId,Age,Sex,Condition
0,0,75 or over,Male,Other
1,1,55-64,Female,Other
2,2,75 or over,Male,Angina Pectoris Prevention
3,3,75 or over,Male,Angina Pectoris Prevention
4,4,75 or over,Male,Angina Pectoris Prevention
...,...,...,...,...
61664,61664,65-74,,Combined High Blood Cholesterol and Triglyceri...
61665,61665,55-64,Male,Low HDL Cholesterol
61666,61666,55-64,Male,High Amount of Triglyceride in the Blood
61667,61667,55-64,Female,Combined High Blood Cholesterol and Triglyceri...


In [22]:
# Data Rating
df_rating = pd.DataFrame({'UserId':df_user['UserId'], 'ItemId':df_drugs['ItemId'], 'Effectiveness':df_drugs['Effectiveness'], 'EaseofUse':df_drugs['EaseofUse'],'Satisfaction':df_drugs['Satisfaction']})
df_rating

,UserId,ItemId,Effectiveness,EaseofUse,Satisfaction
0,0,6054,1,1,1
1,1,6054,5,1,1
2,2,6054,5,5,5
3,3,6054,5,5,5
4,4,6054,5,5,5
...,...,...,...,...,...
61664,61664,93290,5,5,2
61665,61665,93290,4,4,4
61666,61666,93290,2,3,2
61667,61667,93290,3,3,3


## 4. Prapemrosesan

...

## 5. Pembangunan Model Rekomendasi

...


## 6. Pengujian Model Rekomendasi

...

---